# Home work 2 

[Dataset](https://www.kaggle.com/danielgrijalvas/movies/version/2)


## Evaluation

1. Preprocessing of dataset, explaination what column do you use, why, In you skip any column explain why. Please analize usage of [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and/or [TF-IDF Vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).You allow to use external sources. 
2. K-means. Find the best param: number of clusters `n_clusters` (train muptiple models choose the best one). Show that you trained at least two k-means models (train kmeans for dataset with features received by CountVectorizer/TF-IDF Vectorizer and without).
3. Visualize cluster for the best model (2D case, you are allowed to use t-SNE or PCA if you want, not compulsory though)
4. DBScan. Find the best params: epsilon `eps` and minimum number of samples `min_samples`(train muptiple models choose the best one). Show that you trained at least two dbscan models (train dbscan for dataset with features received by CountVectorizer/TF-IDF Vectorizer and without)
5. Visualize clusters for the best model (2D case, you are allowed to use t-SNE or PCA if you want, not compulsory though)
6. Summary for both approaches (describe the model accuracy, performance, score, etc.)

## Additional Info

Sklearn K-means class has property - `inertia_` sum of squared distances of samples to their closest cluster center. Could be used for comparison.

[Here](https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html#sphx-glr-auto-examples-cluster-plot-mini-batch-kmeans-py) is example how you can compare two clustering approaches

[Silhouette score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score)- method of interpretation and validation of consistency within clusters of data [from wiki](https://en.wikipedia.org/wiki/Silhouette_(clustering))

t-SNE - method to reduce the dimensionality down to 2 dimenions, check the [kaggle kernel](https://www.kaggle.com/ffisegydd/cluster-analysis-of-movies-data) for example

## Submit

Two options for submition: via email or on [distedu.ukma.edu.ua](https://distedu.ukma.edu.ua/course/view.php?id=32)

You should submit jupyter notebook by Sunday, November 18th till 11:55 pm EEST timezone.


## Bonus points

Hierarchical clustering. Find best params. Dendogram and explanation.
Try all four merge strategies (complete, average, single linkages and ward criteria)
(8 points)



# Step1: Initialization, reading input


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import time

from scipy import arange
from scipy.cluster.hierarchy import dendrogram, linkage  
from scipy.spatial.distance import pdist
import scipy.cluster.hierarchy as shc
  
from sklearn.cluster import AgglomerativeClustering,DBSCAN,KMeans
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from matplotlib import pyplot as plt
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objs as go

%matplotlib inline

In [ ]:
temp_data = pd.read_csv("movies.csv")


# Step2: Data exploration & preprocessing

### Simple exploration:

In [ ]:
temp_data.info()

No nulls, everything except object-typed values seems OK.<br>
Let's check how many unique values each category has

In [ ]:
temp_data.nunique().plot.bar()

Some features have pretty low amount of unique values (genre, country, year). Others(vote, name, gross) are almost all unique.<br>
Let's look how unique values are distributed among different categories(company/country/genre/rating)

In [ ]:
#"plotting+" func
def show_dist(series,title,threshold):
    temp,cnt,num = pd.Series(),0,0
    #not really effective, but the idea was to create that 'other' category
    for i in series.index:
        if series[i]<threshold:
            num+=series[i]
            cnt+=1
        else:
            temp[i]=series[i]
    temp['Other'] = num
    text = "Index 'Other' contains "+str(cnt)+" indexes"
    plt.figure(1,figsize=(8,8))
    plt.bar(temp.index,temp.values)
    plt.suptitle(title+'\n'+text)
    #plt.xlabel(text)
    plt.xticks(rotation='vertical')
    plt.show()

In [ ]:
categories = [
    #category_name - category_description - threshold
    #threshold is chosen manually
    
    ["company","Movies by company of production:",200],
    ["country","Movies by country of production:",100],
    ["genre","Movies by genre:",100],
    ["rating","Movies by rating:",100]
]
for category in categories:
    naming,title, threshold = category    
    show_dist(temp_data[naming].value_counts(),title,threshold)
#Wasted 2 hours on this
#The next day I found same graphs in the dataset description on Kaggle. Ah, whatever..

As you may see, most features with relatively few categories (i.e. rating, genre) have a "dominant category", which sometimes consists of more elements than all other categories of that feature.<br>
Special category is a year, because all the values there are spread evenly<br>
Some numerical categories (especially, score, can be normalized and transformed into categories with discrete number of values 

In [ ]:
#possibly useful normalization function
def normalize(dataframe,cols= ['budget','gross','runtime','score','votes']):
    temp = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(dataframe[cols]))
    dataframe[cols] = temp
    return dataframe

In [ ]:
#let's test a function. We'll be using: budget,gross,runtime,score,votes
#columns = ['budget','gross','runtime','score']
test_df_normalized = normalize(temp_data)
test_df_normalized.head()
#seems legit

Certain columns should be tf-idf vectorized, others may simply be grouped by<br>
We won't group by 'director','star','writer'.In my opinion, it makes no sense:<br>
- firstly, I don't see any logic behind building tf-idf word matrices by person's separate name and surname. Imagine we have Tony Scott, Ridley Scott and Tony Kaye. Each of them is a director, shares name or surname with another, yet their movies are absolutely different. 
- secondly, there are simply too many unique values. Adding them all will extend our model with thousands of extra features of questionable quality.

In fact, I don't see any sense to group any column by words: let's look at a distribution of words met in a column "company" and then analyze a column "name": 

In [ ]:
def explore_column_by_words(column,threshold=100):
    #adding something to pd.Series ain't efficient at all, but its a good way to generate a new words table step by step
    temp = pd.Series(temp_data[column].str.split())
    #small list of words to ignore
    stopwords = ["the","of","and","in","to","el","la","de","a","&","on","for","2","2:","3","4","5"]#I chose them myself
    words = pd.Series()
    for line in temp:
        for word in line:
            if word.lower() not in stopwords:
                word = re.sub("[^a-zA-Z]","",word)
                if word=="":
                    continue
                if word in words:
                    words[word]+=1
                else:
                    words[word]=1
    print("Total words:", words.count())
    print("Unique words:",words.nunique(),"\n\nWords by frequency:\n")
    print(words.sort_values(ascending=False))
    show_dist(words,"Words by frequency",threshold)


In [ ]:
explore_column_by_words("company")

Most common words are "Pictures", "Films", "Entertainment", etc. Using tf-idf model by this words (in our task and current dataset) is senseless.<br>
Maybe the only suitable column will be 'name'

In [ ]:
#we ignored some of the most typical words and set threshold at 30

explore_column_by_words("name",25)

### Feature engineering: naive approach
Primarily, we'll assume that we don't all text fields except for 'company'
We will generate bool features using get_dummies on 'genre' and 'rating':

In [ ]:
#Preprocessing: cleaning the data set. 
temp_data = pd.read_csv("movies.csv")
#we just drop most of text values
temp_data.released = pd.to_datetime(temp_data.released)
temp_data.drop(['director','name','star','country','writer','released'],axis=1,inplace=True)

In [ ]:
#creating dummies for groups where tf-idf/countvectorizer are pointless
temp_data = pd.concat([temp_data,pd.get_dummies(temp_data.genre.astype('category'))],axis=1)
temp_data = pd.concat([temp_data,pd.get_dummies(temp_data.rating.astype('category'))],axis=1)

temp_data.drop(['genre','rating'],axis=1,inplace=True)

In [ ]:
#data_d - set with no vectorization except for genre&rating
data_d = temp_data.copy()
data_d.drop('company',axis=1,inplace=True)


For the sake of curiosity, we will use tf-idf vectorizer with 'company' (around 2.5k unique words).
We'll create additional dataframe for that. Also, we'll create a dataframe with 'company', vectorized by count-vectorizer. Then we'll run KMeans model on different dataframes and compare results using inertia


In [ ]:
#TF IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    lowercase=True,
    token_pattern=r'\w{1,}',
    stop_words='english',
    max_features=2500,
    min_df = 1)

def tfidf_vectorize_feature(data,feature):
    v = tfidf_vectorizer.fit_transform(data[feature])
    tfidf_vect = pd.DataFrame(v.todense(), columns=tfidf_vectorizer.get_feature_names())
    temp = pd.concat([data, tfidf_vect], axis=1)
    temp.drop(feature,axis=1,inplace=True)
    return temp

In [ ]:
#CountVectorizer
count_vectorizer = CountVectorizer(
    strip_accents='unicode',
    analyzer='word',
    lowercase=True,
    token_pattern=r'\w{1,}',
    stop_words='english',
    max_features=2500,
    min_df = 1)

def count_vectorize_feature(data,feature):
    c = count_vectorizer.fit_transform(data[feature])
    count_vect = pd.DataFrame(c.todense(),columns=count_vectorizer.get_feature_names())
    temp = pd.concat([data,count_vect], axis=1)
    temp.drop(feature,axis=1,inplace=True)
    return temp

In [ ]:
#vectorize single feature
data_t = tfidf_vectorize_feature(temp_data,'company')
data_c = count_vectorize_feature(temp_data,'company')


# Step3: Building K-Means model.

In [ ]:
#K-Means - different num of clusters
def kmeans_model(desc,clusters,data):
    inertia = []
    print(desc)
    for i in range(2,clusters+1):
        model = KMeans(n_clusters=i)
        model.fit(data)
        inertia.append(model.inertia_)
        if i%10==0:
            print("Inertia on {} clusters: {}".format(i,model.inertia_))
    return inertia

### "almost no text features" approach

In [ ]:
#runs ~40mins(i5-7300HQ), inertia on every set is very similar and giant (around 1.3*10^18 for 20 clusters)
stats_d = kmeans_model("Starter, only dummies",50,data_d)#runs in couple minutes
stats_t = kmeans_model("Starter with dummies, tf-idf-ized 'company'",50,data_t)
stats_c = kmeans_model("Starter with dummief, countv-ized 'company'",50,data_c)
#Starter with dummies, tf-idf-ized 'company'
#Inertia on 10 clusters: 2.7484860688811126e+18
#Inertia on 20 clusters: 1.3325447011389642e+18
#Inertia on 30 clusters: 8.614824405600724e+17
#Inertia on 40 clusters: 6.403562908464607e+17
#Inertia on 50 clusters: 5.0286967614973037e+17
#Starter with dummief, countv-ized 'company'
#Inertia on 10 clusters: 2.749849066197541e+18
#Inertia on 20 clusters: 1.3330596587224942e+18
#Inertia on 30 clusters: 8.707042343487918e+17
#Inertia on 40 clusters: 6.274067966952358e+17
#Inertia on 50 clusters: 4.969134127733565e+17

As we can see, performance between tf-idf and countvec doesn't really differ. It doesn't mean they're the same thing. Maybe it means that we chose features poorly

### Feature engineering: everything featurized*

*everything but 'released'

In [ ]:
#data preparation
def prepare_dataset():
    temp_data = pd.read_csv("movies.csv")  
    #temp_data.released = pd.to_datetime(temp_data.released)
    #temp_data[['r_year','r_month','r_day']] = temp_data.released.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))

    #getting dummies from smaller categories
    temp_data = pd.concat([temp_data,pd.get_dummies(temp_data.genre.astype('category'))],axis=1)
    temp_data = pd.concat([temp_data,pd.get_dummies(temp_data.rating.astype('category'))],axis=1)
    #temp_data = pd.concat([temp_data,pd.get_dummies(temp_data.year.astype('category'))],axis=1)    
    #temp_data = pd.concat([temp_data,pd.get_dummies(temp_data.r_year.astype('category'))],axis=1)
    
    temp_data.drop(columns=['released','genre','rating'],axis=1,inplace=True)
    return temp_data

In [ ]:
features = ['company','country','director','name','star','writer']
data_tfd = prepare_dataset()

for f in features:
    data_tfd = tfidf_vectorize_feature(data_tfd,f)

stats_all_vect = kmeans_model("Starter, all features vectorized",50,data_tfd)#don't run - tons of time and shitty results
#10 clusters: 2.748529066997636e+18
#20 clusters: 1.3400117916318536e+18
...
#50 clusters: 5.044264970147402e+17

### more adequate approach:

Now lets repeat 2 previous approaches with normalized features:


In [ ]:
data_tfd_normalized = normalize(prepare_dataset)
for f in features:
    data_tfd_normalized = tfidf_vectorize_feature(data_tfd_normalized,f)


In [ ]:
stats_tfd_normalized = kmeans_model("Normalized set",50,data_tfd_normalized) #runs for another 30 mins.
#Normalized set
#Inertia on 10 clusters: 27963.21365206909
#Inertia on 20 clusters: 26670.18770438666
#Inertia on 30 clusters: 25839.625499081725
#Inertia on 40 clusters: 25523.85550936234
#Inertia on 50 clusters: 25191.060694315413

In [ ]:
#let's try scaler instead of our normalization
data = prepare_dataset()
for f in features:
    data = tfidf_vectorize_feature(data,f)

pipeline = Pipeline([('scale', StandardScaler())])
tfd_scaled = pipeline.fit_transform(data)

In [ ]:
stats_tfd_scaled = kmeans_model("StandardlyScaled set",50,data_tfd_normalized)# almost no difference, don't run
#10 clusters: 27823.169699275626
#20 clusters: 26602.967411147692
#30 clusters: 25853.884082875506
#40 clusters: 25480.932187425915
#50 clusters: 25220.73599407387

Let´s plot them all

In [ ]:
plt.figure(1,figsize=(12,12))
plt.plot(stats_d,label='only dummies')
plt.plot(stats_t,label='only one feature tf-idf-ized')
plt.plot(stats_c,label='only one feature tf-idf-ized')

plt.plot(stats_all_vect,label="all features vectorized")
plt.plot(stats_tfd_normalized,label='all features vectorized and normalized')
plt.plot(stats_tfd_scaled,label='all features vectorized and scaled standardly')

plt.legend()

In [ ]:
plt.figure(1,figsize=(12,12))
plt.plot(stats_tfd_normalized,label='all features vectorized and normalized')
plt.plot(stats_tfd_scaled,label='all features vectorized and scaled standardly')

plt.legend()


In [ ]:
#more dummies, less tf-idf:
data = prepare_dataset()
data = pd.concat([data,pd.get_dummies(data.country.astype('category'))],axis=1)

data.drop(['country','company','name','director','star','writer'],axis=1,inplace=True)
data = normalize(data,cols=['budget','gross','runtime','score','votes','year'])

In [ ]:
stats_c_vec = kmeans_model("No vectorization",50,data)#No vectorization
#Inertia on 10 clusters: 6418.163381197919
#Inertia on 20 clusters: 4566.427523204975
#Inertia on 30 clusters: 3613.2555433390507
#Inertia on 40 clusters: 3071.705614612101
#Inertia on 50 clusters: 2655.380634499413

In [ ]:
plt.plot(stats_no_vec,label="only ´year´,'genre' and 'rating vectorized'")
plt.plot(stats_c_vec,label="also 'company' vectorized")
plt.legend()

Via elbow method, `n_clusters` around 10 seems optimal

In [ ]:
optimal_clusters,optimal_data = 8,tfd_scaled

## Visualization: t-SNE

we'll use plotly

In [ ]:
def perform_tsne(clusters,data):
    model = KMeans(n_clusters=clusters)
    #model.fit(data)
    start = time.time()
    pred = model.fit_predict(data)
    t1 = time.time()-start #seconds to compute pred
    print("model fitted in {} seconds".format(t1))
    tsne = TSNE()#default components num
    start = time.time()
    tsne_fit = tsne.fit_transform(data)
    t2 = time.time()-start#seconds to fit-transform via t-SNE
    print("tsne fitted in {} seconds".format(t2))
    return tsne_fit,pred

In [ ]:
init_notebook_mode(connected=True)
def plot_tsne(name,fit,pred):#pred is for colors
    
    trace = go.Scatter(
        x=fit.T[0], 
        y=fit.T[1],
        mode='markers',
        text=name,
        textposition='top center',
        name='Lines, Markers and Text',
        marker=dict(
            color = pred, 
            colorscale='Portland',
            showscale=True
        )
    )
    
    
    vis_data = [trace]
    layout = go.Layout(
    showlegend=False)
    visualization = go.Figure(data=vis_data, layout=layout)
    iplot(visualization)

"Non-vectorized model". Seems to be OK

In [ ]:
tsne_fit, pred = perform_tsne(9,data)#runs around 3 mins
plot_tsne("test",tsne_fit,pred)

In [ ]:
optimal_clusters,optimal_data = 12,tfd_scaled #from that graph with vectorized features above

In [ ]:
tsne_fit, pred = perform_tsne(optimal_clusters,optimal_data)# runs about 15mins

Every text field splitted with tf-idf by words. Looks awful. 

In [ ]:
plot_tsne("test",tsne_fit,pred)

# Step 4: DBScan

We'll find optimal `eps`, num of `samples` and compare models by `silhouette_score`

In [ ]:
#DBScan - different number of clusters
def dbscan_model(desc,min_eps,min_samples,data):
    sils = []
    print(desc)
    for i in arange(min_eps,min_eps+5,0.2):
        for j in arange(min_samples, min_samples+7,1):
            print("eps={},min_samples={}".format(i,j))
            model = DBSCAN(n_jobs=-1,eps=min_eps+i,min_samples=min_samples+j)
            model = model.fit(data)
            pred = model.fit_predict(data)
            if len(np.unique(model.labels_))>1:
                sil = silhouette_score(data,model.labels_)
                sils.append(sil)
                print("Silhouette score on eps={}, min_samples={} is: {}".format(i,j,sil))
            #if pred[np.argmin(pred)]!=-1:
            #if i%1==0:
            i = round(i,1)
            j = round(j,1)
    return sils

In [ ]:
#simple no-vect dataset
data = prepare_dataset()
data.drop(columns=['company','country','director','name','star','writer'],axis=1,inplace=True)
data=normalize(data)
data=normalize(data,cols=['year'])
data.head()


In [ ]:
stats_dbscan_novec = dbscan_model("No vect model:",0.5,4,data)

In [ ]:
#vect dataset:
data = prepare_dataset()
#let's tf-idf 'company','country'
data = tfidf_vectorize_feature(data,'country')
data = tfidf_vectorize_feature(data,'company')

data=normalize(data)

data=normalize(data,'year')
data.drop(columns=['director','name','star','writer'],axis=1,inplace=True)


In [ ]:
stats_dbscan_vec = dbscan_model("More vect model:",0.5,4,data)

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(stats_dbscan_novec,label = "non-vectorized set")
plt.plot(stats_dbscan_vec,label = "vectorized set")
plt.legend()

In [ ]:
optimal_eps,optimal_samples=0.7,6 #let's say that

## Visualization: t-SNE


In [ ]:
def perform_tsne_dbscan(ep,samples):
    model = DBSCAN(n_jobs=-1,eps=ep,min_samples=samples)
    start = time.time()
    pred = model.fit_predict(data)
    t1 = time.time()-start #seconds to compute pred
    print("model fitted in {} seconds".format(t1))
    tsne = TSNE()#default components num
    start = time.time()
    tsne_fit = tsne.fit_transform(data)
    t2 = time.time()-start#seconds to fit-transform via t-SNE
    print("tsne fitted in {} seconds".format(t2))
    return tsne_fit,pred

In [ ]:
tsne_fit,pred = perform_tsne_dbscan(optimal_eps,optimal_samples)
plot_tsne("non-vectorized",tsne_fit,pred)

# Bonus: Hierarchical clustering

Let's build a dendogram hardly relying on rating:

In [ ]:
#preparing a moderate set of features:
data=pd.read_csv("movies.csv")
data = normalize(data)
data = normalize(data,cols=['year'])
data = pd.concat([data,pd.get_dummies(data.rating.astype('category'))],axis=1)
data.drop(['company','country','director','star','name','writer','genre','rating','released'],inplace=True,axis=1)
data.head()

In [ ]:
Z = linkage(pdist(data.T.values))#single-linkage using scipy.cluster.hierarchy

plt.figure(figsize=(10,10))
dendrogram(orientation='top',Z=Z,distance_sort='descending',show_leaf_counts=True,labels = data.columns)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
def perform_tsne_hierarchical(clusters):
    cluster = AgglomerativeClustering(n_clusters=clusters, affinity='euclidean', linkage='average')  
    start = time.time()
    pred = cluster.fit_predict(data)  
    t1 = time.time()-start #seconds to compute pred
    print("model fitted in {} seconds".format(t1))
    tsne = TSNE()#default components num
    start = time.time()
    tsne_fit = tsne.fit_transform(data)
    t2 = time.time()-start#seconds to fit-transform via t-SNE
    print("tsne fitted in {} seconds".format(t2))
    return tsne_fit,pred

Actually, it's not that bad.

In [ ]:
tsne_fit,pred = perform_tsne_hierarchical(4)
plot_tsne("Hierarchical",tsne_fit,pred)


# Summary

### 1. K-Means
Using sets with different amounts of vectorized features (using both tf-idf and countvectorizer) I showed that tf-idf word vectorizing fields containing lexemes is simply harmful and doesn't lead anywhere. I guess, we should cautiously use text fields from dataset in a more complex model.<br>
Anyway, for different sets of features I built clusters\inertia graphs and chose optimal cluster number using elbow method<br>
I didn't get a good clusterer on "vectorized" version, it may be seen on a graph.<br>
### 2. DBScan
I tried to find optimal eps and min_samples, but got a little confused.<br>
I don't like any of the graphs. Maybe it's about poor feature selection<br>.
### General
I managed to try different techniques and approaches.
In both models I should try selecting features\dummies differently, as well as better search for optimal`eps`,`min_samples`,etc.I guess Hierarchical clustering fits this task better than traditional K-Means.
#### Thanks for the attention!
 
 
 
Following notebook belongs to Alexandr Kroshyn. 